In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# 这里使用的ID为1~N版本
class Person:
  def __init__(self, id):
    self.id = id
    self.alive = True # 是否存活
    self.offers = []  # 收到的offer
    self.job = -1 # 签约的企业

class Recruit:
  def __init__(self, id, num):
    self.id = id
    self.num = num # 岗位剩余容量
    self.workers = [] # 签约的员工
    
class Pair:
  def __init__(self, per, rec, val):
    self.pid = per.id
    self.rid = rec.id
    self.val = val
  
  def print(self):
    print(self.pid, self.rid, self.val)

In [3]:
# 属性嵌入，这里的序号是0~N-1
# 用于拼接向量 and 判断行业
a_emb_job = pd.read_csv('./attribute_embedding_position.txt',sep=' ',index_col=0,header=None,na_values=[])
# emb_job.drop(columns=[0,65],inplace=True)
a_emb_per = pd.read_csv('./attribute_embedding_user.txt',sep=' ',index_col=0,header=None,na_values=[])
# emb_per.drop(columns=[0,220],inplace=True)
print(a_emb_job.shape, a_emb_per.shape)

(1575, 74) (868, 74)


In [41]:
alive = []
for i in a_emb_per.index:
  alive.append(i)
ss = pd.Series(alive)
ss.to_csv('./no_fw.csv')

In [48]:
l = fw_list.shape[0]
sum = 0
map_list = []
for i in range(100):
  if is_fw(i):
    map_list.append(-1)
  else:
    map_list.append(sum)
    sum += 1

# map_list

In [6]:
# 硬核条件，这里的序号是1~N
hd_job = pd.read_csv('./hardcore_position.csv',index_col=0,header=0,encoding='gbk')
hd_per = pd.read_csv('./hardcore_user.csv',index_col=0,header=0,encoding='gbk')
hd_job.shape, hd_per.shape

((1575, 4), (10877, 3))

In [35]:
idx = a_emb_per.index
def is_fw(pid):
  # return not pid in idx
  val = fw_list.iloc[pid,0]
  return val==1

fw_list = pd.read_csv('./fw.txt',sep=' ',index_col=0)

nfw_cnt = 0
for i in range(10877):
  if not is_fw(i):
    nfw_cnt+=1
    print(i,end=' ')

nfw_cnt
# fw_list

0 7 10 12 13 14 15 20 22 23 24 25 26 27 28 29 30 31 32 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 52 54 55 57 59 60 63 64 65 66 68 69 70 71 72 74 75 76 77 79 80 81 82 83 84 85 86 88 89 90 91 92 93 94 96 97 100 101 102 103 104 105 106 107 108 109 110 112 113 114 116 117 118 119 120 121 123 124 126 127 128 129 130 131 132 133 134 138 139 140 141 142 143 144 145 146 147 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 170 172 173 177 178 180 181 182 183 184 185 186 187 188 189 190 191 193 194 195 196 198 199 200 201 202 203 204 205 207 208 209 210 211 212 226 227 237 256 280 308 311 315 325 339 350 363 371 390 425 499 527 548 552 584 591 601 614 617 625 652 659 667 707 715 720 731 760 785 810 812 855 858 869 881 883 889 916 919 931 944 1003 1015 1019 1022 1111 1121 1148 1149 1157 1168 1217 1235 1263 1288 1315 1386 1416 1468 1472 1502 1516 1553 1555 1558 1563 1584 1615 1640 1650 1666 1671 1675 1726 1757 1760 1773 1808 1833 1843 1911 1912 1940 1973 2029 2044

868

In [20]:
is_fw(1584)

False

In [9]:
hd_per.head()

,预期最大薪资,预期最小薪资,工作性质
序号,,,
1,8000,7000,实习
2,10000,6000,全职
3,6000,3000,全职
4,9000,4000,全职
5,5000,3000,全职


In [10]:
hd_job.head()

,性质,岗位需求量,最大薪资,最小薪资
序号,,,,
1,全职,6,20000,12000
2,全职,10,26000,16000
3,全职,100,25000,15000
4,全职,5,8000,5000
5,全职,6,6000,3500


In [ ]:
def get_input_1(pid, rid):
  per = pca_per_1.loc[pid].values
  job = pca_job_1.loc[rid].values
  all = np.concatenate([per,job],axis=0)
  return all

pca_per_1 = pd.read_csv('./pca_per.csv',index_col=0,header=None)
pca_job_1 = pd.read_csv('./pca_job.csv',index_col=0,header=None)

In [31]:
def judge_hangye(pid,rid): # 0~N-1
  # print(pid,rid)
  x = a_emb_per.loc[pid,6:14].values.astype('int')
  y = a_emb_job.loc[rid,6:14].values.astype('int')
  sum = (x*y).sum()
  return sum > 0

def judge_gongzi(pid,rid): # 0~N-1
  p2, p1 = hd_per.iloc[pid,:-1]
  r2, r1 = hd_job.iloc[rid,-2:]
  # print('')
  # print(p1, p2)
  # print(r1, r2)
  return max(p1,r1)<=min(p2,r2)

def judge_type(pid,rid):
  pt = hd_per.iloc[pid,-1]
  rt = hd_job.iloc[rid,0]
  # print(pt,rt)
  return pt == rt

def judge(pid,rid):
  b1 = judge_hangye(pid,rid)
  b2 = judge_gongzi(pid,rid)
  b3 = judge_type(pid,rid)
  return b1 and b2 and b3

In [36]:
# print(judge_hangye(41,2))
true_cnt = 0
false_cnt = 0

for i in tqdm(range(10877)):
  if is_fw(i):
    continue
  for j in range(1575):
    b1 = judge_hangye(i,j)
    b2 = judge_gongzi(i,j)
    b3 = judge_type(i,j)
    res = b1 and b2 and b3
    if not res:
      false_cnt+=1
    else:
      true_cnt+=1
      
      
true_cnt,false_cnt
      
    
# judge_type(0,4)


100%|██████████| 10877/10877 [15:25<00:00, 11.76it/s] 


(140933, 1226167)

In [38]:
140933/(140933+1226167)

0.10308902055445834

In [6]:
df_ppd = pd.read_csv('./p-u-rating.txt',sep=' ')
df_myd = pd.read_csv('./u-p-rating.txt',sep=' ')
df_ppd.head(), df_myd.head()

(   position  user  ppd
 0         0    94    6
 1         0   165    6
 2         0   168    6
 3         1    94    6
 4         1   165    6,
    user  position  myd
 0   425       295    8
 1   425      1504    8
 2   425      1515    8
 3   425      1542    8
 4   425      1543    8)

In [6]:
def get_ppd(per_id, job_id):
  return ppd[per_id-1, job_id-1]

def get_myd(per_id, job_id):
  return myd[per_id-1, job_id-1]

def hardcore(per_id, job_id):
  return True

In [7]:
persons = []
recruits = []

In [17]:
ppd[:30,:30]
pos_cnt = 0
for i in tqdm(range(m_shape[0])):
  for j in range(m_shape[1]):
    if ppd[i][j]==0:
      pos_cnt+=1

pos_cnt

100%|██████████| 10877/10877 [00:06<00:00, 1721.33it/s]


11991089

In [9]:
for i in tqdm(range(df_per.shape[0])):
  id = df_per.loc[i, '序号']
  tmp = Person(id)
  persons.append(tmp)
  # print(id)

100%|██████████| 10877/10877 [00:00<00:00, 79668.07it/s]


In [10]:
for i in tqdm(range(df_rec.shape[0])):
  id = df_rec.loc[i+1, '序号']
  num = df_rec.loc[i+1, '岗位需求量']
  tmp = Recruit(id, num)
  recruits.append(tmp)
  # print(id)

100%|██████████| 1575/1575 [00:00<00:00, 80665.60it/s]


In [11]:
def get_cnt_rec():
  sum = 0
  for i in recruits:
    sum += i.num
  return sum

old_cnt_rec = get_cnt_rec()
old_cnt_rec

5520

In [12]:
epoch = 0
cnt_rec = get_cnt_rec()
while True:
  epoch += 1
  # 招聘回合
  print(f'epoch {epoch}:')
  for rec in tqdm(recruits):  # 对于每个岗位
    # 发放offer
    pair_list = [] # 临时数组：<per, rec, val>
    # 计算与每个求职者的匹配度
    for per in persons:
      val = get_ppd(per.id, rec.id)
      if per.alive==False or val==0:
        continue
      pair = Pair(per, rec, val)
      pair_list.append(pair)
    # 就地排序 by val
    pair_list.sort(reverse=True, key=lambda x:x.val)
    pair_list = pair_list[:rec.num] # 取TopK
    
    for pair in pair_list:
      per = persons[pair.pid-1]
      rec = recruits[pair.rid-1]
      if per.alive:
        per.offers.append(rec.id) # 发放offer
  
  # 所有岗位offer发放完毕
    
  # 求职者签约
  for per in persons: # 对于每个求职者
    if per.alive == False:
      continue
    tmp2list = [] # 临时数组：Pair
    for rid in per.offers:  # 对于每一份offer
      rec = recruits[rid-1]
      val = get_myd(per.id, rec.id)
      pair = Pair(per, rec, val)
      tmp2list.append(pair)
    if len(tmp2list)==0:  # 当前求职者无offer
      continue
    
    tmp2list.sort(reverse=True, key=lambda x:x.val) # 取最高满意度
    # 目标岗位
    dest_rec = recruits[tmp2list[0].rid-1]
    
    if dest_rec.num > 0: # 若未招满
      dest_rec.num -= 1
      per.job = dest_rec.id
      per.alive = False
      per.offers.clear()
      dest_rec.workers.append(per)
  
  new_cnt_rec = get_cnt_rec()
  increase = cnt_rec - new_cnt_rec
  cnt_rec = new_cnt_rec
  print(f'increase {increase}\n')
  if increase <= 0:
    break

epoch 1:


100%|██████████| 1575/1575 [00:20<00:00, 78.70it/s]


increase 4320

epoch 2:


100%|██████████| 1575/1575 [00:16<00:00, 96.16it/s] 


increase 1108

epoch 3:


100%|██████████| 1575/1575 [00:15<00:00, 102.46it/s]


increase 92

epoch 4:


100%|██████████| 1575/1575 [00:15<00:00, 102.63it/s]

increase 0



In [13]:
# for item in pair_list:
#   # print(item.pid, item.rid, item.val)
#   item.print()
st = set()
cnt_pair = 0
for rec in recruits: # 对于每个岗位
  print(f'岗位{rec.id}, 招到{len(rec.workers)}, 还差{rec.num}' ,end='\t: ')
  if len(rec.workers)>0:  # 若非空
    
    for per in rec.workers: # 对于每个签约求职者
      print(f'<{per.id},{get_ppd(per.id,rec.id)}>', end=' ')
      cnt_pair += 1
      st.add(per.id)
  print('')
    
print(cnt_pair, len(st))

岗位1, 招到6, 还差0	: <2299,0.9999596434373103> <7623,0.9997591344547964> <9482,0.9999434459880182> <10598,0.9998405650458714> <10754,0.999802968338296> <4426,0.999030080956119> 
岗位2, 招到10, 还差0	: <3785,0.9993846353482438> <4005,0.999778344503349> <4794,0.9994810382150876> <4903,0.9998217773508467> <5626,0.9994027113457316> <7342,0.999845942035411> <193,0.9989853980700225> <5187,0.9988609500896569> <7510,0.9989438094124538> <9521,0.9989536851180553> 
岗位3, 招到100, 还差0	: <393,0.9940963091604456> <428,0.9952575067374336> <520,0.9955462846054455> <647,0.9943695182246071> <797,0.9971816479424574> <863,0.994962265555129> <1024,0.9978338911079712> <1325,0.9996665350953242> <1333,0.9972625623196554> <1359,0.9969149610934422> <1363,0.9913182760490902> <1415,0.991962135083142> <1726,0.9904484369972378> <1728,0.996261023499532> <1794,0.9982879191893009> <1890,0.9981476750253692> <2062,0.9996163131653598> <2103,0.9992861285028151> <2177,0.993866437592309> <2221,0.9974241101642585> <2243,0.9978113619022659

In [14]:
obj = cnt_pair/old_cnt_rec
obj

1.0